In [1]:
import json
import requests
import pandas as pd
import importlib
from requests_oauthlib import OAuth2Session

In [2]:
token=open('token.json',mode='r')
token=token.read()
token

'{"access_token": "ya29.GlseBQ2EHC3X2VD9UO_TO_h1u61c9DvacFJMfNFKz54clAT3Q-6kCB-ux754CGh5wY7K3o8Ajln1xmgJjxcxBuUGT2caGGrk2HwwR_Cmz46ydSH8wQ0ON2WJhiPd", "expires_in": 3600, "token_type": "Bearer", "expires_at": 1512884856.271626, "refresh_token": "1/4oXPMNa6MQBPwqIerHCyZgMF3H1iW2dFbTwmTHX9RRqK-pfiakClwdNTpjixbdE1"}'

In [3]:
token=json.loads(token)
token

{'access_token': 'ya29.GlseBQ2EHC3X2VD9UO_TO_h1u61c9DvacFJMfNFKz54clAT3Q-6kCB-ux754CGh5wY7K3o8Ajln1xmgJjxcxBuUGT2caGGrk2HwwR_Cmz46ydSH8wQ0ON2WJhiPd',
 'expires_at': 1512884856.271626,
 'expires_in': 3600,
 'refresh_token': '1/4oXPMNa6MQBPwqIerHCyZgMF3H1iW2dFbTwmTHX9RRqK-pfiakClwdNTpjixbdE1',
 'token_type': 'Bearer'}

In [4]:
import keys
importlib.reload(keys)
keychain = keys.keychain
client_id = keychain['youtube']['client_id']
scope = keychain['youtube']['scope']
redirect = keychain['youtube']['redirect_uris'][0]
refresh_url = keychain['youtube']['token_uri']
session = OAuth2Session(client_id, scope=scope, redirect_uri=redirect)
keychain['youtube']

{'auth_provider_x509_cert_url': 'https://www.googleapis.com/oauth2/v1/certs',
 'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
 'client_id': '126920892433-e58m7pn87600743drvlsjsanlcsus2av.apps.googleusercontent.com',
 'client_secret': 'flFfvozgUappY1MIRCPE5Cxc',
 'project_id': 'project5-187402',
 'redirect_uris': ['urn:ietf:wg:oauth:2.0:oob', 'http://localhost'],
 'scope': ['https://www.googleapis.com/auth/youtube.readonly'],
 'token_uri': 'https://accounts.google.com/o/oauth2/token'}

In [22]:
def getData(data,vid,country):
    if country != "" and "CountryCode" not in data:
        data["CountryCode"] = []
    for item in vid:
        data['videoId'].append(item['id'])
        data['channelId'].append(item['snippet']['channelId'])
        data['categoryId'].append(item['snippet']['categoryId'])
        data['channelTitle'].append(item['snippet']['channelTitle'])
        data['publishedAt'].append(int(item['snippet']['publishedAt'][11:13]))
        if country != "":
            data['CountryCode'].append(country)
    return data

In [8]:
def vid_by_country(token,country=""):
    D={}
    D['access_token'] = token['access_token']
    D['part'] = ['snippet,contentDetails,statistics']
    D['maxResults'] = '50'
    D['chart'] ='mostPopular'
    D['PageToken'] = 'CAUQAA'
    if country != "":
        D["regionCode"] = country
    url ='https://www.googleapis.com/youtube/v3/videos'
    popular_vid = requests.get(url, params = D)
    popular_vid = popular_vid.json()
    
    page = []
    data = {'videoId':[], 'channelId':[],'categoryId':[] ,'channelTitle':[], 'publishedAt':[]}
    for i in range(10):
        if 'nextPageToken' not in popular_vid:
            next_page = 'CAUQAA'
        else:
            next_page = popular_vid['nextPageToken']
        page.append(next_page)
        D['PageToken'] = next_page
        popular_vid = requests.get(url, params = D)
        popular_vid = popular_vid.json()
        getData(data,popular_vid['items'],country)
        
    df= pd.DataFrame(data)
    return df

df_global = vid_by_country(token)
df_global.to_csv('df_global.csv')
df_global[:10]

,categoryId,channelId,channelTitle,publishedAt,videoId
0,24,UCqFzWxSCi39LnW1JKFR3efg,Saturday Night Live,08,A8HLnDP6uRM
1,23,UC67f2Qf7FYhtoUIF4Sf29cA,Bad Lip Reading,00,S-4rhjO6xYg
2,24,UC8JE00xTMBOqKs7o0grFTfQ,Poppy,01,ZWAkzvL5Ho4
3,22,UCbAwSkqJ1W_Eg7wr3cp5BUA,Safiya Nygaard,05,Ep6-bNu16_I
4,24,UCZGYJFUizSax-yElQaFDp5Q,Star Wars,02,UVNIEszp2UQ
5,24,UCWOA1ZGywLbqmigxE4Qlvuw,Netflix,21,QK_iX5cPDhE
6,23,UCPcFg7aBbaVzXoIKSNqwaww,jacksfilms,23,X3gKYTc6lbw
7,24,UC0v-tlzsn0QZwJnkiaUSJVQ,FBE,22,fuHcdFPBKy0
8,23,UCQG4cX86zZ51IU2cerZgPSA,Awkward Puppets,17,bo8OYPYRTIY
9,10,UCGIelM2Dj3zza3xyV3pL3WQ,MigosVEVO,02,9v_rtaye2yY


In [26]:
df_US = vid_by_country(token,"US")
df_US.to_csv('df_US.csv')

In [25]:
df_France = vid_by_country(token,'FR')
df_France.to_csv('df_France.csv')

In [27]:
df_GB = vid_by_country(token,'GB')
df_GB.to_csv('df_GB.csv')

In [28]:
df_Spain = vid_by_country(token,"ES")
df_Spain.to_csv('df_Spain.csv')

In [ ]:
# D={}
# D['access_token'] = token['access_token']
# D['part'] = ['snippet,contentDetails,statistics']
# D['maxResults'] = '50'
# D['chart'] ='mostPopular'
# D['PageToken'] = 'CAUQAA'
# D['regionCode'] = 'FR'
# url ='https://www.googleapis.com/youtube/v3/videos'
# popular_vid = requests.get(url, params = D)
# popular_vid = popular_vid.json()

# data = {'videoId':[], 'channelId':[],'categoryId':[] ,'channelTitle':[], 'publishedAt':[]}
# getData(data,popular_vid['items'],'FR')
# df = pd.DataFrame(data)
# df

In [32]:
url = 'https://www.googleapis.com/youtube/v3/videoCategories'
C = {}
C['access_token'] = token['access_token']
C['part'] = 'snippet'
C['regionCode'] = 'US'
category_vid = requests.get(url, params = C)
category_vid = category_vid.json()

In [33]:
data_category = {'id':[],'title':[]}
def categoryTable(data,json):
    for item in json:
        data['id'].append(item['id'])
        data['title'].append(item['snippet']['title'])
categoryTable(data_category,category_vid['items'])
df1 = pd.DataFrame(data_category)
df1[:10]

,id,title
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports
5,18,Short Movies
6,19,Travel & Events
7,20,Gaming
8,21,Videoblogging
9,22,People & Blogs


In [ ]:
result = pd.merge(df_global, df1, left_on='categoryId', right_on='id')
result.to_csv('result.csv')

In [52]:
###############################################
# update
country = pd.concat([df_US, df_France, df_GB, df_Spain], ignore_index=True)
country.to_csv('country.csv')
country[:10]

,CountryCode,categoryId,channelId,channelTitle,publishedAt,videoId
0,US,24,UCqFzWxSCi39LnW1JKFR3efg,Saturday Night Live,8,A8HLnDP6uRM
1,US,23,UC67f2Qf7FYhtoUIF4Sf29cA,Bad Lip Reading,0,S-4rhjO6xYg
2,US,24,UC8JE00xTMBOqKs7o0grFTfQ,Poppy,1,ZWAkzvL5Ho4
3,US,22,UCbAwSkqJ1W_Eg7wr3cp5BUA,Safiya Nygaard,5,Ep6-bNu16_I
4,US,24,UCZGYJFUizSax-yElQaFDp5Q,Star Wars,2,UVNIEszp2UQ
5,US,24,UCWOA1ZGywLbqmigxE4Qlvuw,Netflix,21,QK_iX5cPDhE
6,US,23,UCPcFg7aBbaVzXoIKSNqwaww,jacksfilms,23,X3gKYTc6lbw
7,US,24,UC0v-tlzsn0QZwJnkiaUSJVQ,FBE,22,fuHcdFPBKy0
8,US,23,UCQG4cX86zZ51IU2cerZgPSA,Awkward Puppets,17,bo8OYPYRTIY
9,US,10,UCGIelM2Dj3zza3xyV3pL3WQ,MigosVEVO,2,9v_rtaye2yY


In [53]:
###############################################
# update
country_cate =  pd.merge(country, df1, left_on='categoryId', right_on='id')
country_cate.to_csv('country_cate.csv')
country_cate[:10]

,CountryCode,categoryId,channelId,channelTitle,publishedAt,videoId,id,title
0,US,24,UCqFzWxSCi39LnW1JKFR3efg,Saturday Night Live,8,A8HLnDP6uRM,24,Entertainment
1,US,24,UC8JE00xTMBOqKs7o0grFTfQ,Poppy,1,ZWAkzvL5Ho4,24,Entertainment
2,US,24,UCZGYJFUizSax-yElQaFDp5Q,Star Wars,2,UVNIEszp2UQ,24,Entertainment
3,US,24,UCWOA1ZGywLbqmigxE4Qlvuw,Netflix,21,QK_iX5cPDhE,24,Entertainment
4,US,24,UC0v-tlzsn0QZwJnkiaUSJVQ,FBE,22,fuHcdFPBKy0,24,Entertainment
5,US,24,UCMtFAi84ehTSYSE9XoHefig,The Late Show with Stephen Colbert,8,00nmxR1mxIA,24,Entertainment
6,US,24,UCGCVyTWogzQ4D170BLy2Arw,Simply Nailogical,16,ta6sV6WrElE,24,Entertainment
7,US,24,UCq0OueAsdxH6b8nyAspwViw,Universal Pictures,1,vn9mMeWcgoM,24,Entertainment
8,US,24,UCSAUGyc_xA8uYzaIVG6MESQ,nigahiga,19,JBDWIGpKnwM,24,Entertainment
9,US,24,UCKy1dAqELo0zrOtPkf0eTMw,IGN,20,QsLmJq8cHlA,24,Entertainment


In [47]:
country_cate =  pd.merge(country, df1, left_on='categoryId', right_on='id')
country_cate.to_csv('country_cate.csv')
pie = country_cate.groupby(['CountryCode','title']).count()[['videoId']]
country_cate
pie.reset_index(inplace = True)
pie
pie.to_csv('pie.csv')

In [24]:
token = session.refresh_token(refresh_url, 
                             client_id=keychain['youtube']['client_id'],
                             client_secret=keychain['youtube']['client_secret'],
                             refresh_token=token['refresh_token'])